## Table of Contents
- [Data Structures](#data-structures)
- [Memory Usage](#memory-usage)
- [Panel Data](#panel-data)
- [Time Series](#time-series)
- [Reshaping Structure](#reshaping-the-dataset-structure)
- [Selecting Data](#selecting-data)
- [Tidy Data](#tidy-data)
- [Long Format](#long-format)
- [Wide Format](#wide-format)

---
## Data Structures
`Cross-sectional` data is collected at a single point in time, capturing information across various subjects to analyze patterns and relationships among variables at that moment.

`Time-series data` is collected over different time periods for the same subject, allowing the tracking of changes and trends over time.

`Panel (or longitudinal) data` combines cross-sectional and time-series elements by collecting data on the same subjects across multiple time periods, enabling analysis of both time-based changes and cross-sectional differences.

---
## Memory Usage
Define a function to calculate memory usage for DataFrames

In [2]:
# print memory usage of a dataframe in MB
def memory_usage(df):
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.3f} MB")

---
## Panel Data

### Download data
As an example panel data take a look at Statistics Canada, Labour Force Survey, [Table 14-10-0287-01](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1410028701), Labour force characteristics, monthly, seasonally adjusted and trend-cycle.

In [3]:
import pandas as pd
import urllib.parse

# This script demonstrates how to download a CSV file from the Statistics Canada website
# using the downloadDbLoadingData-nonTraduit.action endpoint.
base_url = "https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action"
pid = "1410028701"
latestN =  ""
startDate = "20190101"
endDate = ""
csvLocale = "en"
selected_members = "[[1,2,3,4,5,6,7,8,9,10,11],[1,2,3,4,5,6,7,8,9],[1,2,3],[1,2,6,7],[1],[1,2]]"

#encode selected members
encoded_selected_members = urllib.parse.quote(selected_members,safe='')

# Construct the full URL
url = "{}?pid={}&latestN={}&startDate={}&endDate={}&csvLocale={}\
&selectedMembers={}".format(base_url,pid,latestN,startDate,endDate,
csvLocale,encoded_selected_members)

# Print the URL
print(url)

# Read the CSV data into a DataFrame
df_1 = pd.read_csv(url)

# Display the DataFrame and its memory usage
print(df_1.info())
print(df_1.shape)
memory_usage(df_1)
df_1.tail()

https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid=1410028701&latestN=&startDate=20190101&endDate=&csvLocale=en&selectedMembers=%5B%5B1%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%5D%2C%5B1%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%5D%2C%5B1%2C2%2C3%5D%2C%5B1%2C2%2C6%2C7%5D%2C%5B1%5D%2C%5B1%2C2%5D%5D


C:\Users\mehdi\AppData\Local\Temp\ipykernel_11572\2781864947.py:26: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1 = pd.read_csv(url)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170236 entries, 0 to 170235
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   REF_DATE                      170236 non-null  object 
 1   GEO                           170236 non-null  object 
 2   DGUID                         170236 non-null  object 
 3   Labour force characteristics  170236 non-null  object 
 4   Gender                        170236 non-null  object 
 5   Age group                     170236 non-null  object 
 6   Statistics                    170236 non-null  object 
 7   Data type                     170236 non-null  object 
 8   UOM                           170236 non-null  object 
 9   UOM_ID                        170236 non-null  int64  
 10  SCALAR_FACTOR                 170236 non-null  object 
 11  SCALAR_ID                     170236 non-null  int64  
 12  VECTOR                        170236 non-nul

,REF_DATE,GEO,DGUID,Labour force characteristics,Gender,Age group,Statistics,Data type,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
170231,2024-09,British Columbia,2021A000259,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066966,11.9.3.7.1.2,31.3,NaN,NaN,NaN,1
170232,2024-10,British Columbia,2021A000259,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066966,11.9.3.7.1.2,30.7,NaN,NaN,NaN,1
170233,2024-11,British Columbia,2021A000259,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066966,11.9.3.7.1.2,28.9,NaN,NaN,NaN,1
170234,2024-12,British Columbia,2021A000259,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066966,11.9.3.7.1.2,29.4,NaN,NaN,NaN,1
170235,2025-01,British Columbia,2021A000259,Employment rate,Women+,55 years and over,Estimate,Unadjusted,Percent,239,units,0,v2066966,11.9.3.7.1.2,28.7,NaN,NaN,NaN,1


### Remove extra columns
Removing extra columns reduced the memory usage to half the downloaded data.

In [6]:
# drop extra columns
columns_to_drop = ['DGUID', 'COORDINATE', 'Statistics', 'UOM', 'SCALAR_FACTOR',
'UOM_ID','SCALAR_ID','VECTOR', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']

df_2 = df_1.drop(columns=[col for col in columns_to_drop if col in df_1.columns], axis=1)

# Display the DataFrame and its memory usage
memory_usage(df_2)
df_2.tail()

Memory usage: 60.661 MB


,REF_DATE,GEO,Labour force characteristics,Gender,Age group,Data type,VALUE
170231,2024-09,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,31.3
170232,2024-10,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,30.7
170233,2024-11,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,28.9
170234,2024-12,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,29.4
170235,2025-01,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,28.7


---
## Time Series

### Add date index
First step in creating a time series is to add a DatetimeIndex

In [7]:
# convert the 'REF_DATE' column to a datetime object
df_2['REF_DATE']=pd.to_datetime(df_2['REF_DATE'], format='%Y-%m')

# set the 'REF_DATE' column as the index
df_2.set_index('REF_DATE', inplace=True)

# convert the index to a period index
df_2.index = df_2.index.to_period('M')

# Display the DataFrame and its memory usage
memory_usage(df_2)
df_2.tail()


Memory usage: 52.868 MB


,GEO,Labour force characteristics,Gender,Age group,Data type,VALUE
REF_DATE,,,,,,
2024-09,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,31.3
2024-10,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,30.7
2024-11,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,28.9
2024-12,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,29.4
2025-01,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,28.7


### Pivot Table
* Each column in the following table is a time series.
* Pivot table transforms the dataset from the long to wide format and create columns index.
* Memory usage of pivot table (long format) is **1/50** of wide format (original data) due to removal of information duplication in text columns.
* The wide format is hard for human interpretation, but efficient for processing and storage.
* Now we only have numbers (facts) inside the table and all dimensions are moved to row and column labels

In [8]:
# pivot from long to wide format
df_3 = df_2.pivot_table(index=['REF_DATE'],
columns=['Data type','GEO','Gender','Age group','Labour force characteristics'], 
values='VALUE')

# Display the DataFrame and its memory usage
memory_usage(df_3)
print(df_3.info())
print(df_3.shape)
df_3.tail()

Memory usage: 1.299 MB
<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 73 entries, 2019-01 to 2025-01
Freq: M
Columns: 2332 entries, ('Seasonally adjusted', 'Alberta', 'Men+', '15 to 24 years', 'Employment') to ('Unadjusted', 'Saskatchewan', 'Women+', '55 years and over', 'Unemployment rate')
dtypes: float64(2332)
memory usage: 1.3 MB
None
(73, 2332)


Data type                    Seasonally adjusted                  \
GEO                                      Alberta                   
Gender                                      Men+                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE                                                           
2024-09                                    165.9            53.8   
2024-10                                    168.1            54.3   
2024-11                                    174.0            55.9   
2024-12                                    174.4            55.9   
2025-01                                    175.7            56.1   

Data type                                                       \
GEO                                                              
Gender                                                           
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE                                                         
2024-09                                     106.1        196.6   
2024-10                                     106.4        203.9   
2024-11                                     111.2        202.3   
2024-12                                     109.8        206.7   
2025-01                                     106.4        207.5   

Data type                                                             \
GEO                                                                    
Gender                                                                 
Age group                                                              
Labour force characteristics Part-time employment Participation rate   
REF_DATE                                                               
2024-09                                      59.9               63.8   
2024-10                                      61.7               65.8   
2024-11                                      62.7               65.0   
2024-12                                      64.6               66.2   
2025-01                                      69.3               66.2   

Data type                                                               \
GEO                                                                      
Gender                                                                   
Age group                                                                
Labour force characteristics Population Unemployment Unemployment rate   
REF_DATE                                                                 
2024-09                           308.3         30.7              15.6   
2024-10                           309.7         35.7              17.5   
2024-11                           311.0         28.3              14.0   
2024-12                           312.2         32.3              15.6   
2025-01                           313.3         31.8              15.3   

Data type                                       ...        Unadjusted  \
GEO                                             ...      Saskatchewan   
Gender                                          ...            Women+   
Age group                    15 years and over  ...    25 to 54 years   
Labour force characteristics        Employment  ... Unemployment rate   
REF_DATE                                        ...                     
2024-09                                 1343.3  ...               4.7   
2024-10                                 1349.5  ...               4.1   
2024-11                                 1364.1  ...               3.3   
2024-12                                 1377.3  ...               4.2   
2025-01                                 1378.4  ...               3.8   

Data type                                                       \
GEO                                           

### MultiIndex columns
* Pivot tables create a multi-index for columns

In [10]:
# columns is a multiindex
print(type(df_3.columns),'\n')
for i in range(0, len(df_3.columns.names)):
    print(f'Level {i}: {df_3.columns.levels[i]}\n')

<class 'pandas.core.indexes.multi.MultiIndex'> 

Level 0: Index(['Seasonally adjusted', 'Unadjusted'], dtype='object', name='Data type')

Level 1: Index(['Alberta', 'British Columbia', 'Canada', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Nova Scotia', 'Ontario',
       'Prince Edward Island', 'Quebec', 'Saskatchewan'],
      dtype='object', name='GEO')

Level 2: Index(['Men+', 'Total - Gender', 'Women+'], dtype='object', name='Gender')

Level 3: Index(['15 to 24 years', '15 years and over', '25 to 54 years',
       '55 years and over'],
      dtype='object', name='Age group')

Level 4: Index(['Employment', 'Employment rate', 'Full-time employment', 'Labour force',
       'Part-time employment', 'Participation rate', 'Population',
       'Unemployment', 'Unemployment rate'],
      dtype='object', name='Labour force characteristics')



---
## Reshaping the dataset structure

### Stacking
* Stack and unstack can be used to move data labels freely between coulmns and rows

In [11]:
#.stack() rotates the lowest level of the column MultiIndex to the row index
df_3.stack().tail()


C:\Users\mehdi\AppData\Local\Temp\ipykernel_6152\3849219951.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_3.stack().tail()


Data type                             Seasonally adjusted                    \
GEO                                               Alberta                     
Gender                                               Men+                     
Age group                                  15 to 24 years 15 years and over   
REF_DATE Labour force characteristics                                         
2025-01  Part-time employment                        69.3             151.6   
         Participation rate                          66.2              73.9   
         Population                                 313.3            2003.5   
         Unemployment                                31.8             102.0   
         Unemployment rate                           15.3               6.9   

Data type                                                               \
GEO                                                                      
Gender                                                                   
Age group                             25 to 54 years 55 years and over   
REF_DATE Labour force characteristics                                    
2025-01  Part-time employment                   48.0               NaN   
         Participation rate                     93.1              44.8   
         Population                           1069.1             621.1   
         Unemployment                           55.8              14.4   
         Unemployment rate                       5.6               5.2   

Data type                                                               \
GEO                                                                      
Gender                                Total - Gender                     
Age group                             15 to 24 years 15 years and over   
REF_DATE Labour force characteristics                                    
2025-01  Part-time employment                  166.9             469.4   
         Participation rate                     65.0              68.8   
         Population                            608.1            4002.0   
         Unemployment                           52.7             183.7   
         Unemployment rate                      13.3               6.7   

Data type                                                               \
GEO                                                                      
Gender                                                                   
Age group                             25 to 54 years 55 years and over   
REF_DATE Labour force characteristics                                    
2025-01  Part-time employment                  196.0             106.6   
         Participation rate                     87.7              39.4   
         Population                           2112.5            1281.4   
         Unemployment                          108.0              23.0   
         Unemployment rate                       5.8               4.6   

Data type                                                               ...  \
GEO                                                                     ...   
Gender                                        Women+                    ...   
Age group                             15 to 24 years 15 years and over  ...   
REF_DATE Labour force characteristics                                   ...   
2025-01  Part-time employment                   97.6             317.8  ...   
         Participation rate                     63.6              63.6  ...   
         Population                            294.8            1998.5  ...   
         Unemployment                           20.9              81.8  ...   
         Unemployment rate                      11.1               6.4  ...   

Data type                                 Unadjusted                    \
GEO                                     Saskatchewan                     
Gender                                         

In [12]:
#Stacking will move the lowest level of the column MultiIndex to the
# row index and creates a multilevel row index
df_3.stack().index

C:\Users\mehdi\AppData\Local\Temp\ipykernel_6152\249213413.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_3.stack().index


MultiIndex([('2019-01',           'Employment'),
            ('2019-01',      'Employment rate'),
            ('2019-01', 'Full-time employment'),
            ('2019-01',         'Labour force'),
            ('2019-01', 'Part-time employment'),
            ('2019-01',   'Participation rate'),
            ('2019-01',           'Population'),
            ('2019-01',         'Unemployment'),
            ('2019-01',    'Unemployment rate'),
            ('2019-02',           'Employment'),
            ...
            ('2024-12',    'Unemployment rate'),
            ('2025-01',           'Employment'),
            ('2025-01',      'Employment rate'),
            ('2025-01', 'Full-time employment'),
            ('2025-01',         'Labour force'),
            ('2025-01', 'Part-time employment'),
            ('2025-01',   'Participation rate'),
            ('2025-01',           'Population'),
            ('2025-01',         'Unemployment'),
            ('2025-01',    'Unemployment rate')],
   

In [13]:
# We can also pass in an argument to select the level we would like to stack
df_3.stack(level=2).tail()

C:\Users\mehdi\AppData\Local\Temp\ipykernel_6152\2151654873.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_3.stack(level=2).tail()


Data type                    Seasonally adjusted                  \
GEO                                      Alberta                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE Gender                                                    
2024-12  Total - Gender                    340.3            56.1   
         Women+                            166.0            56.5   
2025-01  Men+                              175.7            56.1   
         Total - Gender                    342.3            56.3   
         Women+                            166.6            56.5   

Data type                                                       \
GEO                                                              
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE Gender                                                  
2024-12  Total - Gender                     188.5        394.7   
         Women+                              78.7        188.0   
2025-01  Men+                               106.4        207.5   
         Total - Gender                     175.4        395.0   
         Women+                              69.0        187.5   

Data type                                                             \
GEO                                                                    
Age group                                                              
Labour force characteristics Part-time employment Participation rate   
REF_DATE Gender                                                        
2024-12  Total - Gender                     151.9               65.1   
         Women+                              87.3               64.0   
2025-01  Men+                                69.3               66.2   
         Total - Gender                     166.9               65.0   
         Women+                              97.6               63.6   

Data type                                                               \
GEO                                                                      
Age group                                                                
Labour force characteristics Population Unemployment Unemployment rate   
REF_DATE Gender                                                          
2024-12  Total - Gender           606.1         54.4              13.8   
         Women+                   293.9         22.1              11.8   
2025-01  Men+                     313.3         31.8              15.3   
         Total - Gender           608.1         52.7              13.3   
         Women+                   294.8         20.9              11.1   

Data type                                       ...        Unadjusted  \
GEO                                             ...      Saskatchewan   
Age group                    15 years and over  ...    25 to 54 years   
Labour force characteristics        Employment  ... Unemployment rate   
REF_DATE Gender                                 ...                     
2024-12  Total - Gender                 2572.7  ...               4.5   
         Women+                         1195.4  ...               4.2   
2025-01  Men+                           1378.4  ...               5.0   
         Total - Gender                 2568.4  ...               4.4   
         Women+                         1190.1  ...               3.8   

Data type                                                       \
GEO                                                              
Age group                    55 years and over                   
Labour force characteristics        Employment Employment rate   
REF_DATE Gender                                                  
2024-12  Total - Gender                  127.9            37.6   
         Women+                           56.4            32.1   
202

In [14]:
# multiple levels can be stacked at once
df_3.stack(level=[1,2]).tail()

C:\Users\mehdi\AppData\Local\Temp\ipykernel_6152\3537842986.py:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_3.stack(level=[1,2]).tail()


Data type                            Seasonally adjusted                  \
Age group                                 15 to 24 years                   
Labour force characteristics                  Employment Employment rate   
REF_DATE GEO          Gender                                               
2025-01  Quebec       Total - Gender               614.7            61.5   
                      Women+                       309.0            63.5   
         Saskatchewan Men+                          42.0            54.1   
                      Total - Gender                81.4            54.7   
                      Women+                        39.4            55.4   

Data type                                                               \
Age group                                                                
Labour force characteristics         Full-time employment Labour force   
REF_DATE GEO          Gender                                             
2025-01  Quebec       Total - Gender                281.6        676.6   
                      Women+                        117.8        341.4   
         Saskatchewan Men+                           30.4         48.6   
                      Total - Gender                 49.9         92.2   
                      Women+                         19.5         43.5   

Data type                                                                     \
Age group                                                                      
Labour force characteristics         Part-time employment Participation rate   
REF_DATE GEO          Gender                                                   
2025-01  Quebec       Total - Gender                333.1               67.6   
                      Women+                        191.2               70.2   
         Saskatchewan Men+                           11.6               62.6   
                      Total - Gender                 31.5               62.0   
                      Women+                         19.9               61.2   

Data type                                                     \
Age group                                                      
Labour force characteristics         Population Unemployment   
REF_DATE GEO          Gender                                   
2025-01  Quebec       Total - Gender     1000.2         61.9   
                      Women+              486.4         32.3   
         Saskatchewan Men+                 77.6          6.7   
                      Total - Gender      148.7         10.8   
                      Women+               71.1          4.1   

Data type                                                                 ...  \
Age group                                              15 years and over  ...   
Labour force characteristics         Unemployment rate        Employment  ...   
REF_DATE GEO          Gender                                              ...   
2025-01  Quebec       Total - Gender               9.1            4635.6  ...   
                      Women+                       9.5            2216.9  ...   
         Saskatchewan Men+                        13.8             324.0  ...   
                      Total - Gender              11.7             606.4  ...   
                      Women+                       9.4             282.4  ...   

Data type                                   Unadjusted                    \
Age group                               25 to 54 years 55 years and over   
Labour force characteristics         Unemployment rate        Employment   
REF_DATE GEO          Gender                                               
2025-01  Quebec       Total - Gender               5.1             959.1   
                      Women+                       4.2             442.0   
         Saskatchewan Men+                         5.0              68.5   
                      Total - Gender               4.4             123.2   
            

### Transpose
* Transpose can make a wide format readable
* Date are now displayed in columns
* This is similar to Statistics Canada beyond 20/20 format
* YOY and MOM changes can be easily calculated

In [19]:
# transpose the data to make it easier to view
df_3.transpose()

REF_DATE                                                                                2019-01  \
Data type           GEO          Gender Age group         Labour force characteristics            
Seasonally adjusted Alberta      Men+   15 to 24 years    Employment                      152.9   
                                                          Employment rate                  58.0   
                                                          Full-time employment            102.0   
                                                          Labour force                    173.1   
                                                          Part-time employment             50.9   
...                                                                                         ...   
Unadjusted          Saskatchewan Women+ 55 years and over Part-time employment             19.0   
                                                          Participation rate               35.9   
                                                          Population                      162.5   
                                                          Unemployment                      2.2   
                                                          Unemployment rate                 3.8   

REF_DATE                                                                                2019-02  \
Data type           GEO          Gender Age group         Labour force characteristics            
Seasonally adjusted Alberta      Men+   15 to 24 years    Employment                      151.1   
                                                          Employment rate                  57.3   
                                                          Full-time employment             92.9   
                                                          Labour force                    174.6   
                                                          Part-time employment             58.2   
...                                                                                         ...   
Unadjusted          Saskatchewan Women+ 55 years and over Part-time employment             18.9   
                                                          Participation rate               35.4   
                                                          Population                      162.7   
                                                          Unemployment                      2.0   
                                                          Unemployment rate                 3.5   

REF_DATE                                                                                2019-03  \
Data type           GEO          Gender Age group         Labour force characteristics            
Seasonally adjusted Alberta      Men+   15 to 24 years    Employment                      152.9   
                                                          Employment rate                  58.0   
                                                          Full-time employment             99.6   
                                                          Labour force                    173.9   
                                                          Part-time employment             53.4   
...                                                                                         ...   
Unadjusted          Saskatchewan Women+ 55 years and over Part-time employment             19.3   
                                                          Participation rate               36.8   
                                                          Population                      163.0   
                                                          Unemployment                      2.3   
                                                          Unemployment rate                 3.8   

REF_DATE                                                                                2019-04  \
Data type           GEO          Gend

---
## Selecting Data

### Selecting by loc

In [20]:
# select rows by index label
df_3.loc['2023':'2024']

Data type                    Seasonally adjusted                  \
GEO                                      Alberta                   
Gender                                      Men+                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE                                                           
2023-01                                    152.8            55.3   
2023-02                                    156.1            56.3   
2023-03                                    155.2            55.7   
2023-04                                    154.0            55.0   
2023-05                                    150.7            53.6   
2023-06                                    153.0            54.1   
2023-07                                    155.9            54.9   
2023-08                                    153.8            53.8   
2023-09                                    145.1            50.5   
2023-10                                    155.1            53.7   
2023-11                                    155.2            53.4   
2023-12                                    158.0            54.1   
2024-01                                    158.1            53.8   
2024-02                                    160.5            54.3   
2024-03                                    151.8            51.1   
2024-04                                    161.4            54.0   
2024-05                                    158.4            52.6   
2024-06                                    159.1            52.5   
2024-07                                    163.7            53.7   
2024-08                                    166.6            54.3   
2024-09                                    165.9            53.8   
2024-10                                    168.1            54.3   
2024-11                                    174.0            55.9   
2024-12                                    174.4            55.9   

Data type                                                       \
GEO                                                              
Gender                                                           
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE                                                         
2023-01                                      97.4        173.2   
2023-02                                      99.8        177.4   
2023-03                                     100.6        177.9   
2023-04                                      95.4        176.3   
2023-05                                      92.9        173.9   
2023-06                                      87.0        178.3   
2023-07                                      91.8        181.2   
2023-08                                      99.3        179.3   
2023-09                                      83.0        173.2   
2023-10                                      82.4        177.7   
2023-11                                      90.6        185.4   
2023-12                                      92.7        184.8   
2024-01                                      92.8        186.0   
2024-02                                      99.0        190.5   
2024-03                                      92.0        181.4   
2024-04                                     102.5        194.0   
2024-05                                      79.4        187.5   
2024-06                                     104.9        185.9   
2024-07                                     106.1        187.3   
2024-08                                     109.6        194.5   
2024-09                                     106.1        196.6   
2024-10                                     106.4        203.9   
2024-11                                     111.2        202.3   
2024-12                                     1

In [26]:
# suubset the data on rows and a single column
df_3.loc['2024', ('Seasonally adjusted', 'Canada','Total - Gender', 
                '15 years and over','Employment')]

REF_DATE
2024-01    20577.1
2024-02    20607.7
2024-03    20614.5
2024-04    20700.5
2024-05    20698.3
2024-06    20715.9
2024-07    20712.9
2024-08    20742.6
2024-09    20779.3
2024-10    20782.6
2024-11    20826.4
2024-12    20917.4
Freq: M, Name: (Seasonally adjusted, Canada, Total - Gender, 15 years and over, Employment), dtype: float64

In [27]:
# Header helps to know the selected time series
df_3.loc['2024', [('Seasonally adjusted', 'Canada','Total - Gender', 
                '15 years and over','Employment')]]

Data type,Seasonally adjusted
GEO,Canada
Gender,Total - Gender
Age group,15 years and over
Labour force characteristics,Employment
REF_DATE,
2024-01,20577.1
2024-02,20607.7
2024-03,20614.5
2024-04,20700.5
2024-05,20698.3


In [29]:
# Selecting two columms
df_3.loc['2024', [('Seasonally adjusted', 'Canada','Men+', 
                '15 years and over','Employment'),
                  ('Seasonally adjusted', 'Canada','Women+', 
                '15 years and over','Employment')
                  ]]

Data type                    Seasonally adjusted                  
GEO                                       Canada                  
Gender                                      Men+            Women+
Age group                      15 years and over 15 years and over
Labour force characteristics          Employment        Employment
REF_DATE                                                          
2024-01                                  10857.7            9719.4
2024-02                                  10876.6            9731.1
2024-03                                  10867.5            9747.0
2024-04                                  10944.0            9756.5
2024-05                                  10921.6            9776.7
2024-06                                  10917.3            9798.6
2024-07                                  10923.9            9789.0
2024-08                                  10932.8            9809.8
2024-09                                  10950.4            9828.9
2024-10                                  10964.1            9818.5
2024-11                                  10998.7            9827.7
2024-12                                  11059.3            9858.0

### Passing a set of conditions to LOC

In [31]:
# A practical way to select columns is to create a set of conditions
# eaach condition can use bitwise operators to combine multiple conditions
cond_1 = df_3.columns.get_level_values('GEO') == 'Canada'
cond_2 = (df_3.columns.get_level_values('Labour force characteristics') == 'Employment') | \
(df_3.columns.get_level_values('Labour force characteristics') == 'Unemployment')
cond_3 = df_3.columns.get_level_values('Data type') == 'Unadjusted'
cond_4 = df_3.columns.get_level_values('Age group') == '15 years and over'
cond_5 = df_3.columns.get_level_values('Gender') == 'Total - Gender'

#use loc to select the data
df_3.loc['2024', cond_1 & cond_2 & cond_3 & cond_4 & cond_5]


Data type                           Unadjusted             
GEO                                     Canada             
Gender                          Total - Gender             
Age group                    15 years and over             
Labour force characteristics        Employment Unemployment
REF_DATE                                                   
2024-01                                20246.6       1314.5
2024-02                                20369.9       1309.7
2024-03                                20411.5       1407.6
2024-04                                20583.5       1370.4
2024-05                                20925.4       1399.6
2024-06                                21048.5       1383.4
2024-07                                20915.6       1505.9
2024-08                                20887.4       1682.5
2024-09                                20844.0       1334.2
2024-10                                20861.8       1327.7
2024-11                                20828.6       1445.3
2024-12                                20843.6       1369.7

### Selecting by xs

In [32]:
# cross-sections can be used to select data at a specific level of a MultiIndex
# one value for each level
df_3.xs(level='GEO', key='Canada', axis=1).tail()

Data type                    Seasonally adjusted                  \
Gender                                      Men+                   
Age group                         15 to 24 years                   
Labour force characteristics          Employment Employment rate   
REF_DATE                                                           
2024-09                                   1368.0            53.1   
2024-10                                   1379.2            53.4   
2024-11                                   1379.7            53.3   
2024-12                                   1375.1            53.0   
2025-01                                   1399.7            53.8   

Data type                                                       \
Gender                                                           
Age group                                                        
Labour force characteristics Full-time employment Labour force   
REF_DATE                                                         
2024-09                                     780.0       1615.2   
2024-10                                     795.5       1615.8   
2024-11                                     797.6       1635.3   
2024-12                                     800.7       1634.9   
2025-01                                     831.6       1637.7   

Data type                                                             \
Gender                                                                 
Age group                                                              
Labour force characteristics Part-time employment Participation rate   
REF_DATE                                                               
2024-09                                     588.0               62.7   
2024-10                                     583.7               62.5   
2024-11                                     582.0               63.1   
2024-12                                     574.5               63.0   
2025-01                                     568.2               63.0   

Data type                                                               \
Gender                                                                   
Age group                                                                
Labour force characteristics Population Unemployment Unemployment rate   
REF_DATE                                                                 
2024-09                          2577.4        247.2              15.3   
2024-10                          2584.8        236.6              14.6   
2024-11                          2590.2        255.6              15.6   
2024-12                          2595.4        259.8              15.9   
2025-01                          2599.4        237.9              14.5   

Data type                                       ...        Unadjusted  \
Gender                                          ...            Women+   
Age group                    15 years and over  ...    25 to 54 years   
Labour force characteristics        Employment  ... Unemployment rate   
REF_DATE                                        ...                     
2024-09                                10950.4  ...               5.1   
2024-10                                10964.1  ...               5.3   
2024-11                                10998.7  ...               5.4   
2024-12                                11059.3  ...               4.9   
2025-01                                11091.2  ...               5.5   

Data type                                                       \
Gender                                                           
Age group                    55 years and over                   
Labour force characteristics        Employment Employment rate   
REF_DATE                                                         
2024-09                                 1973.5            29.9   
2024-10                                 1971.3            29.8   
202

In [33]:
# we can filter on multiple levels
df_3.xs(level=('GEO', 
'Labour force characteristics'), key=('Canada', 'Employment'), axis=1)

Data type Seasonally adjusted                                   \
Gender                   Men+                                    
Age group      15 to 24 years 15 years and over 25 to 54 years   
REF_DATE                                                         
2019-01                1332.8            9956.3         6369.9   
2019-02                1332.1            9963.0         6369.8   
2019-03                1323.1            9969.4         6376.1   
2019-04                1363.0           10034.8         6377.5   
2019-05                1318.5           10026.4         6412.8   
...                       ...               ...            ...   
2024-09                1368.0           10950.4         7217.9   
2024-10                1379.2           10964.1         7214.6   
2024-11                1379.7           10998.7         7247.0   
2024-12                1375.1           11059.3         7277.4   
2025-01                1399.7           11091.2         7305.0   

Data type                                                                    \
Gender                      Total - Gender                                    
Age group 55 years and over 15 to 24 years 15 years and over 25 to 54 years   
REF_DATE                                                                      
2019-01              2253.7         2604.5           18920.5        12246.5   
2019-02              2261.0         2618.5           18965.7        12269.0   
2019-03              2270.2         2598.6           18932.9        12234.6   
2019-04              2294.3         2660.9           19053.7        12263.3   
2019-05              2295.2         2617.6           19067.3        12311.6   
...                     ...            ...               ...            ...   
2024-09              2364.5         2686.2           20779.3        13765.5   
2024-10              2370.4         2704.4           20782.6        13759.8   
2024-11              2372.0         2700.8           20826.4        13801.4   
2024-12              2406.8         2693.3           20917.4        13842.4   
2025-01              2386.5         2723.8           20993.4        13905.7   

Data type                                                     ...  \
Gender                              Women+                    ...   
Age group 55 years and over 15 to 24 years 15 years and over  ...   
REF_DATE                                                      ...   
2019-01              4069.5         1271.8            8964.2  ...   
2019-02              4078.2         1286.3            9002.7  ...   
2019-03              4099.7         1275.6            8963.5  ...   
2019-04              4129.6         1297.9            9019.0  ...   
2019-05              4138.0         1299.2            9040.9  ...   
...                     ...            ...               ...  ...   
2024-09              4327.6         1318.2            9828.9  ...   
2024-10              4318.4         1325.3            9818.5  ...   
2024-11              4324.2         1321.1            9827.7  ...   
2024-12              4381.7         1318.2            9858.0  ...   
2025-01              4364.0         1324.0            9902.2  ...   

Data type     Unadjusted                                                     \
Gender              Men+                   Total - Gender                     
Age group 25 to 54 years 55 years and over 15 to 24 years 15 years and over   
REF_DATE                                                                      
2019-01           6268.3            2208.6         2427.6           18579.8   
2019-02           6278.7            2224.9         2457.0           18673.8   
2019-03           6293.9            2246.5         2459.1           18674.2   
2019-04           6328.0            2275.2         2521.9           18875.6   
2019-05           6445.1            2307.7         2712.0           19253.1   
...                  ...               ...            ...               ... 

---
## Tidy Data
Tidy data is defined as data that has the following properties:
* each row corresponds to an observation
* each variable is a column
* each type of observation is in a different table

Also note that Tidy data is a simple flat table with hierarchical index in rows or columns.

In our example, the process of creating tidy data is similar to creting a pivot_table, but dimensions are initially added to row index. 


In [39]:
# pivot from long to wide format, reset index
df_4 = df_2.pivot_table(index=['REF_DATE','Data type','GEO','Gender','Age group'],
columns=['Labour force characteristics'], 
values='VALUE').reset_index()

# Remove columns name
df_4.columns.name = None

# set the 'REF_DATE' column as the index
df_4.set_index('REF_DATE', inplace=True)

# convert the index to a period index
# df_4.index = df_4.index.to_period('M')

# Display the DataFrame and its memory usage
memory_usage(df_4)
print(df_4.info())
print(df_4.shape)
df_4.tail()

Memory usage: 5.990 MB
<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 19272 entries, 2019-01 to 2025-01
Freq: M
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Data type             19272 non-null  object 
 1   GEO                   19272 non-null  object 
 2   Gender                19272 non-null  object 
 3   Age group             19272 non-null  object 
 4   Employment            19272 non-null  float64
 5   Employment rate       19272 non-null  float64
 6   Full-time employment  17666 non-null  float64
 7   Labour force          19272 non-null  float64
 8   Part-time employment  17666 non-null  float64
 9   Participation rate    19272 non-null  float64
 10  Population            19272 non-null  float64
 11  Unemployment          19270 non-null  float64
 12  Unemployment rate     19270 non-null  float64
dtypes: float64(9), object(4)
memory usage: 2.1+ MB
None
(19272, 13)


,Data type,GEO,Gender,Age group,Employment,Employment rate,Full-time employment,Labour force,Part-time employment,Participation rate,Population,Unemployment,Unemployment rate
REF_DATE,,,,,,,,,,,,,
2025-01,Unadjusted,Saskatchewan,Total - Gender,55 years and over,123.2,36.2,90.3,129.2,32.9,37.9,340.8,6.0,4.6
2025-01,Unadjusted,Saskatchewan,Women+,15 to 24 years,37.4,52.6,16.8,41.4,20.5,58.2,71.1,4.0,9.7
2025-01,Unadjusted,Saskatchewan,Women+,15 years and over,282.0,59.1,206.2,294.9,75.9,61.8,477.3,12.9,4.4
2025-01,Unadjusted,Saskatchewan,Women+,25 to 54 years,189.9,82.4,154.1,197.3,35.8,85.6,230.4,7.4,3.8
2025-01,Unadjusted,Saskatchewan,Women+,55 years and over,54.7,31.1,35.2,56.2,19.5,31.9,175.9,1.5,2.7


---
## Long Format
* In the long format, each row is one time point per subject.
* The original Statistics Canada csv file is in the long format.

In [10]:
# drop extra columns
columns_to_drop = ['DGUID', 'Statistics', 'UOM', 'SCALAR_FACTOR',
'UOM_ID','SCALAR_ID','VECTOR', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']

df_5 = df_1.drop(columns=[col for col in columns_to_drop if col in df_1.columns], axis=1)

# Display the DataFrame and its memory usage
memory_usage(df_5)
print(df_5.shape)
df_5.tail()

Memory usage: 70.432 MB
(170236, 8)


,REF_DATE,GEO,Labour force characteristics,Gender,Age group,Data type,COORDINATE,VALUE
170231,2024-09,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,11.9.3.7.1.2,31.3
170232,2024-10,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,11.9.3.7.1.2,30.7
170233,2024-11,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,11.9.3.7.1.2,28.9
170234,2024-12,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,11.9.3.7.1.2,29.4
170235,2025-01,British Columbia,Employment rate,Women+,55 years and over,Unadjusted,11.9.3.7.1.2,28.7


We may drop all dimensions for clarity and just keep the COORDINATE. Each coordinate is a subject. Each row is a subject at a single point of time.

In [36]:
# drop extra columns
columns_to_drop = ['DGUID', 'Statistics', 'UOM', 'SCALAR_FACTOR',
'UOM_ID','SCALAR_ID','VECTOR', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS',
'GEO','Labour force characteristics','Gender','Age group','Data type'
]

df_6 = df_1.drop(columns=[col for col in columns_to_drop if col in df_1.columns], axis=1)

# Display the DataFrame and its memory usage
memory_usage(df_6)
print('Shape:', df_6.shape)
df_6.tail()
# print(df_6.info())

Memory usage: 20.161 MB
Shape: (170236, 3)


,REF_DATE,COORDINATE,VALUE
170231,2024-09,11.9.3.7.1.2,31.3
170232,2024-10,11.9.3.7.1.2,30.7
170233,2024-11,11.9.3.7.1.2,28.9
170234,2024-12,11.9.3.7.1.2,29.4
170235,2025-01,11.9.3.7.1.2,28.7


---
## Wide Format
* In the wide format, a subject’s repeated responses will be in a single row, and each response is in a separate column.

### Step 1 - Collection of time series
* First we need to pivot the Subject ID (COORDINATE)
* This format is interesting in itself, for example we might get a column from dataset and apply an average directly, also all the metadata are stored in the column label.This is a perfect candidate for coumnar storage ia Parquet files.

In [42]:
df_7 = df_6.pivot_table(index='REF_DATE',columns=['COORDINATE'],
    values='VALUE')

# Display the DataFrame and its memory usage
memory_usage(df_7)
print('Shape:', df_7.shape)
df_7.tail()

# df_7.info()

Memory usage: 1.303 MB
Shape: (73, 2332)


COORDINATE,1.1.1.1.1.1,1.1.1.1.1.2,1.1.1.2.1.1,1.1.1.2.1.2,1.1.1.6.1.1,1.1.1.6.1.2,1.1.1.7.1.1,1.1.1.7.1.2,1.1.2.1.1.1,1.1.2.1.1.2,...,9.9.2.7.1.1,9.9.2.7.1.2,9.9.3.1.1.1,9.9.3.1.1.2,9.9.3.2.1.1,9.9.3.2.1.2,9.9.3.6.1.1,9.9.3.6.1.2,9.9.3.7.1.1,9.9.3.7.1.2
REF_DATE,,,,,,,,,,,,,,,,,,,,,
2024-09,34089.8,34089.8,4963.0,4963.0,16486.0,16486.0,12640.8,12640.8,16953.2,16953.2,...,43.7,45.4,59.3,59.3,56.1,51.7,81.0,81.5,32.3,33.1
2024-10,34163.7,34163.7,4975.6,4975.6,16528.9,16528.9,12659.2,12659.2,16991.4,16991.4,...,43.2,44.2,58.9,59.3,55.5,53.1,81.0,81.8,31.5,32.3
2024-11,34225.2,34225.2,4984.9,4984.9,16564.3,16564.3,12676.1,12676.1,17023.0,17023.0,...,42.3,42.5,59.1,59.0,54.7,52.1,81.3,81.6,31.8,32.1
2024-12,34285.1,34285.1,4993.3,4993.3,16599.7,16599.7,12692.1,12692.1,17053.5,17053.5,...,42.9,43.3,59.4,59.7,56.4,54.9,81.4,82.2,31.9,32.1
2025-01,34340.8,34340.8,4999.6,4999.6,16632.1,16632.1,12709.0,12709.0,17081.4,17081.4,...,42.7,41.5,59.2,59.1,55.4,52.6,81.7,82.4,31.2,31.1


### Step 2 - Transpose the collection of time series
* This format is similar to Statistics Canada, Beyond 20/20 tables.

In [43]:
# transpose collection of time series
df_8 = df_7.transpose()

# Display the DataFrame and its memory usage
memory_usage(df_8)
print('Shape:', df_8.shape)
print(df_8.tail())

Memory usage: 1.496 MB
Shape: (2332, 73)
REF_DATE     2019-01  2019-02  2019-03  2019-04  2019-05  2019-06  2019-07  \
COORDINATE                                                                   
9.9.3.2.1.2     55.0     56.0     56.6     55.8     61.0     64.3     63.5   
9.9.3.6.1.1     80.0     80.3     80.0     80.0     80.5     80.4     79.5   
9.9.3.6.1.2     80.2     80.3     80.5     80.7     81.5     81.1     77.2   
9.9.3.7.1.1     34.7     34.5     34.8     35.9     34.7     34.8     35.7   
9.9.3.7.1.2     34.5     34.2     35.4     35.9     34.6     35.3     34.5   

REF_DATE     2019-08  2019-09  2019-10  ...  2024-04  2024-05  2024-06  \
COORDINATE                              ...                              
9.9.3.2.1.2     64.5     57.0     58.0  ...     52.2     60.5     62.7   
9.9.3.6.1.1     80.0     80.1     80.2  ...     81.2     81.2     81.1   
9.9.3.6.1.2     77.5     80.6     80.9  ...     81.8     82.0     81.9   
9.9.3.7.1.1     35.8     35.9     35.8  ..